In [2]:
import numpy as np  
import matplotlib.pyplot as plt  
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

In [4]:
# 2. 복소수 함수 정의
def f(z):
    return np.square(z) - 1

In [5]:
# 3. 함수 테스트: f(0) 계산
print(f(0))  # 예상 출력: (-1+0j)

-1


In [6]:
# 4. 복소수 리스트에 대해 함수 f(z) 적용
z = np.array([4, 1 - 0.2j, 1.6])  # 4, 1-0.2j, 1.6 복소수 리스트
result = f(z)

In [7]:
# 5. 결과 출력
print("입력값:", z)
print("결과값:", result)

입력값: [4. +0.j  1. -0.2j 1.6+0.j ]
결과값: [15.  +0.j  -0.04-0.4j  1.56+0.j ]


In [8]:
# 6. 결과 해석
for z_val, result_val in zip(z, result):
    print(f"입력값 {z_val}에 대한 출력: {result_val}")

입력값 (4+0j)에 대한 출력: (15+0j)
입력값 (1-0.2j)에 대한 출력: (-0.040000000000000036-0.4j)
입력값 (1.6+0j)에 대한 출력: (1.5600000000000005+0j)


In [9]:
# 3. 복소수 집합 생성
# x축과 y축 값을 -10에서 10 사이에서 균일한 간격으로 20개의 값으로 생성
x, y = np.meshgrid(np.linspace(-10, 10, 20), np.linspace(-10, 10, 20))

In [10]:
# 복소수 집합 생성: 실수부 x와 허수부 y를 이용하여 복소수 mesh를 만듦
mesh = x + 1j * y

In [11]:
# 4. 함수 f(z)를 한 번 적용하여 결과를 절대값으로 계산
output = np.abs(f(mesh))

In [12]:
# 5. 3D 산점도로 한 번 적용한 결과 시각화
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 산점도 그리기
ax.scatter(x, y, output, alpha=0.5)

# 레이블과 제목 설정
ax.set_xlabel('Real axis')      # 실수부
ax.set_ylabel('Imaginary axis') # 허수부
ax.set_zlabel('Absolute value') # 함수값의 절대값
ax.set_title('One Iteration: $f(z) = z^2 - 1$')

Text(0.5, 0.92, 'One Iteration: $f(z) = z^2 - 1$')

In [13]:
# 6. 두 번의 함수 반복 적용
output = np.abs(f(f(mesh)))

# 7. 3D 산점도로 두 번 적용한 결과 시각화
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 산점도 그리기
ax.scatter(x, y, output, alpha=0.5)

# 레이블과 제목 설정
ax.set_xlabel('Real axis')      # 실수부
ax.set_ylabel('Imaginary axis') # 허수부
ax.set_zlabel('Absolute value') # 함수값의 절대값
ax.set_title('Two Iterations: $f(z) = z^2 - 1$')

Text(0.5, 0.92, 'Two Iterations: $f(z) = z^2 - 1$')

In [13]:
# 8. 결과 출력
plt.show()

In [14]:
# 3. 유사한 값들 선택
selected_values = np.array([0.4 + 0.4j, 0.41 + 0.4j, 0.4 + 0.41j])
num_iter = 9  # 반복 횟수

# 4. 출력 배열 초기화
outputs = np.zeros(
    (num_iter + 1, selected_values.shape[0]),  # 10X3 Zero 배열 생성
    dtype=complex  # 데이터 타입: 복소수
)
outputs[0] = selected_values   # 함수 초기 조건

# 5. 함수 반복 적용 및 저장
for i in range(num_iter):
    outputs[i + 1] = f(outputs[i])  # 10번 함수 반복하는 동안 각 결과를 저장

# 6. 각 복소수에 대해 반복 적용을 시각화
fig, axes = plt.subplots(1, selected_values.shape[0], figsize=(16, 6))
axes[1].set_xlabel('Real axis')
axes[0].set_ylabel('Imaginary axis')

for ax, data in zip(axes, outputs.T):
    cycle = ax.scatter(
        data.real,  # 실수부
        data.imag,  # 허수부
        c=range(data.shape[0]),  # 반복 횟수를 색으로 표시
        alpha=0.6
    )
    ax.set_title(f'Mapping of iterations on {data[0]}')

fig.colorbar(cycle, ax=axes, location='bottom', label='Iteration')

In [15]:
# 7. 반경(radius)을 초과하면 반복을 중지하는 방식으로 변경
radius_threshold = 2  # 반경 임계값
max_iter = 100  # 최대 반복 횟수

# 8. 반복 횟수 추적 배열 초기화
iteration_counts = np.zeros(selected_values.shape[0])

# 9. 반복 횟수 계산
for i, val in enumerate(selected_values):
    z = val
    for j in range(max_iter):
        if np.abs(z) > radius_threshold:  # 임계값을 초과하면 반복 중지
            iteration_counts[i] = j
            break
        z = f(z)

print(f"Iterations before divergence (threshold > {radius_threshold}):")
print(iteration_counts)

Iterations before divergence (threshold > 2):
[ 8.  7. 11.]


In [16]:
def divergence_rate(mesh, num_iter=10, radius=2):
    z = mesh.copy()  # 복소수 그리드 복사
    diverge_len = np.zeros(mesh.shape)  # 발산 전 반복 횟수를 저장할 배열
    
    # 함수 반복 적용
    for i in range(num_iter):
        # [+] 1. Boolean 배열 마스크 생성: 절대값이 반경보다 작은 값들에 대해 True
        conv_mask = np.abs(z) < radius
        
        # [+] 2. 해당 값들에 대한 diverge_len 값 증가
        diverge_len[conv_mask] += 1
        
        # [+] 3. 해당 값들에 대해 함수 적용: f(z) = z^2 - 1
        z[conv_mask] = z[conv_mask]**2 - 1

    return diverge_len

In [17]:
# 복소수 그리드 생성
x, y = np.meshgrid(np.linspace(-2, 2, 400), np.linspace(-2, 2, 400))
mesh = x + (1j * y)  # x + iy 형태의 복소수 그리드

# 발산 전 반복 횟수 계산
output = divergence_rate(mesh)

# 시각화
fig = plt.figure(figsize=(5, 5))
ax = plt.axes()

ax.set_title('$f(z) = z^2 - 1$')
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')

# 발산 속도에 따른 이미지 출력
im = ax.imshow(output, extent=[-2, 2, -2, 2])
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.1)

# 색상 바 추가
plt.colorbar(im, cax=cax, label='Number of iterations')

plt.show()